# Maltesers: Data Collection | Corporate Project
*HAYA Energy Solutions X IE School of Science & Technology*

*9th July, 2024*

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Initial-Setup" data-toc-modified-id="Initial-Setup-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Initial Setup</a></span></li></ul></li><li><span><a href="#Gas-Prices-Analysis" data-toc-modified-id="Gas-Prices-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Gas Prices Analysis</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Loading-and-Initial-Exploration-of-the-Dataset" data-toc-modified-id="Loading-and-Initial-Exploration-of-the-Dataset-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Loading and Initial Exploration of the Dataset</a></span></li><li><span><a href="#Preprocessing-the-Data" data-toc-modified-id="Preprocessing-the-Data-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Preprocessing the Data</a></span></li><li><span><a href="#Removing-Unnecessary-Columns" data-toc-modified-id="Removing-Unnecessary-Columns-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Removing Unnecessary Columns</a></span></li><li><span><a href="#Checking-for-Missing-Values" data-toc-modified-id="Checking-for-Missing-Values-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Checking for Missing Values</a></span></li><li><span><a href="#Reloading-and-Combining-Historical-and-Recent-Datasets" data-toc-modified-id="Reloading-and-Combining-Historical-and-Recent-Datasets-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>Reloading and Combining Historical and Recent Datasets</a></span></li><li><span><a href="#Extracting-Data-for-2023-2024" data-toc-modified-id="Extracting-Data-for-2023-2024-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>Extracting Data for 2023-2024</a></span></li></ul></li></ul></li><li><span><a href="#Electricity-Prices-Analysis" data-toc-modified-id="Electricity-Prices-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Electricity Prices Analysis</a></span><ul class="toc-item"><li><span><a href="#Electricity-Prices-Analysis-I:-REE" data-toc-modified-id="Electricity-Prices-Analysis-I:-REE-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Electricity Prices Analysis I: REE</a></span><ul class="toc-item"><li><span><a href="#Fetching-and-Combining-Demand-and-Price-Data" data-toc-modified-id="Fetching-and-Combining-Demand-and-Price-Data-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Fetching and Combining Demand and Price Data</a></span></li><li><span><a href="#Loading-and-Displaying-Combined-Demand-and-Price-Data" data-toc-modified-id="Loading-and-Displaying-Combined-Demand-and-Price-Data-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Loading and Displaying Combined Demand and Price Data</a></span></li></ul></li><li><span><a href="#Electricity-Prices-Analysis-II:-OMIE" data-toc-modified-id="Electricity-Prices-Analysis-II:-OMIE-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Electricity Prices Analysis II: OMIE</a></span><ul class="toc-item"><li><span><a href="#Cleaning-Electricity-Price-Data" data-toc-modified-id="Cleaning-Electricity-Price-Data-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Cleaning Electricity Price Data</a></span></li></ul></li></ul></li><li><span><a href="#Weather-Data-Analysis" data-toc-modified-id="Weather-Data-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Weather Data Analysis</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Reading-and-Filtering-Weather-Data" data-toc-modified-id="Reading-and-Filtering-Weather-Data-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Reading and Filtering Weather Data</a></span></li></ul></li></ul></li></ul></div>

### Initial Setup

In [7]:
#Importing Libraries
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import requests
from io import StringIO
from OMIEData.DataImport.omie_marginalprice_importer import OMIEMarginalPriceFileImporter
from OMIEData.Enums.all_enums import DataTypeInMarginalPriceFile
import openmeteo_requests
import requests_cache
from retry_requests import retry
from nixtla import NixtlaClient
import json
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar

## Gas Prices Analysis

This section focuses on the collection and analysis of gas prices using the MIBGAS (Iberian Gas Market) dataset. The dataset includes daily records of gas prices and volumes in the Spanish market. The objective is to clean, preprocess, and merge historical and recent gas price data, ensuring that the data is formatted correctly for subsequent analysis and visualization.

#### Loading and Initial Exploration of the Dataset
First, we load the MIBGAS dataset from a CSV file and conduct an initial exploration to understand its structure. The dataset includes columns for the delivery day, MIBGAS price index, and MIBGAS volume.

In [41]:
file_path = '../data/gas-prices.csv'
df_gas = pd.read_csv(file_path)

df_gas.head()

,Día de entrega,Índice de precio MIBGAS - ES,Volumen MIBGAS - ES
0,01/06/23,25.68,358025
1,02/06/23,24.47,357750
2,03/06/23,23.81,316048
3,04/06/23,23.86,328630
4,05/06/23,24.15,492406


#### Preprocessing the Data
Next, we rename the columns for better readability and convert the `Delivery Day` column to a datetime format. We also ensure that the numeric columns are correctly interpreted as numbers. This step is crucial for accurate data analysis and visualization.

In [44]:
# Rename columns for better readability
df_gas.columns = ['Delivery Day', 'MIBGAS Price Index - ES', 'MIBGAS Volume - ES']

# Convert 'Delivery Day' to datetime format
df_gas['Delivery Day'] = pd.to_datetime(df_gas['Delivery Day'], format='%d/%m/%y')

# Ensure numeric columns are correctly formatted
df_gas['MIBGAS Price Index - ES'] = pd.to_numeric(df_gas['MIBGAS Price Index - ES'], errors='coerce')
df_gas['MIBGAS Volume - ES'] = pd.to_numeric(df_gas['MIBGAS Volume - ES'], errors='coerce')

# Display the first few rows after preprocessing
df_gas.head()

,Delivery Day,MIBGAS Price Index - ES,MIBGAS Volume - ES
0,2023-06-01,25.68,358025
1,2023-06-02,24.47,357750
2,2023-06-03,23.81,316048
3,2023-06-04,23.86,328630
4,2023-06-05,24.15,492406


#### Removing Unnecessary Columns
In this step, we drop the `MIBGAS Volume - ES` column as it is not required for our current analysis.

In [10]:
df_gas23= df_gas.drop(columns='MIBGAS Volume - ES',axis=1)
df_gas23

,Delivery Day,MIBGAS Price Index - ES
0,2023-06-01,25.68
1,2023-06-02,24.47
2,2023-06-03,23.81
3,2023-06-04,23.86
4,2023-06-05,24.15
...,...,...
361,2024-05-27,34.70
362,2024-05-28,34.57
363,2024-05-29,34.42
364,2024-05-30,34.32


#### Checking for Missing Values
We check the dataset for any missing values to ensure data integrity before proceeding with further analysis.

In [11]:
df_gas.isnull().sum()

Delivery Day               0
MIBGAS Price Index - ES    0
MIBGAS Volume - ES         0
dtype: int64

#### Reloading and Combining Historical and Recent Datasets
Since we received these datasets from the company HAYA Energy Solutions, here, we reload the historical and recent gas price datasets, ensuring that the data is correctly formatted and combined into a single dataframe. We also filter out any overlapping data points to avoid duplication.

In [45]:
# Load the historical and recent datasets
file_path_2019 = '../data/Gas_SP_MIBGAS_SpotD_2019-2020.xlsx'
file_path_2023 = '../data/Gas_SP_MIBGAS_SpotD_2023.xlsx'

df_gas2019 = pd.read_excel(file_path_2019, skiprows=1)
df_gas2023 = pd.read_excel(file_path_2023, skiprows=1)

# Rename columns for consistency
df_gas2019.rename(columns={'Dates': 'Delivery Day', 'Values': 'MIBGAS Price Index - ES'}, inplace=True)
df_gas2023.rename(columns={'Dates': 'Delivery Day', 'Values': 'MIBGAS Price Index - ES'}, inplace=True)

# Ensure 'Delivery Day' is in datetime format
df_gas['Delivery Day'] = pd.to_datetime(df_gas['Delivery Day'], format='%d/%m/%Y')
df_gas2019['Delivery Day'] = pd.to_datetime(df_gas2019['Delivery Day'])
df_gas2023['Delivery Day'] = pd.to_datetime(df_gas2023['Delivery Day'])

# Filter out overlapping dates
existing_dates = df_gas['Delivery Day'].dt.date.unique()
df_gas2023 = df_gas2023[~df_gas2023['Delivery Day'].dt.date.isin(existing_dates)]

# Ensure numeric columns are correctly formatted
df_gas['MIBGAS Price Index - ES'] = pd.to_numeric(df_gas['MIBGAS Price Index - ES'], errors='coerce')
df_gas2019['MIBGAS Price Index - ES'] = pd.to_numeric(df_gas2019['MIBGAS Price Index - ES'], errors='coerce')
df_gas2023['MIBGAS Price Index - ES'] = pd.to_numeric(df_gas2023['MIBGAS Price Index - ES'], errors='coerce')

# Drop unnecessary columns
df_gas2019 = df_gas2019.loc[:, ['Delivery Day', 'MIBGAS Price Index - ES']]
df_gas2023 = df_gas2023.loc[:, ['Delivery Day', 'MIBGAS Price Index - ES']]

# Combine the datasets
df_mibgas = pd.concat([df_gas2019, df_gas2023, df_gas], ignore_index=True)

# Save the combined dataset to a CSV file
# df_mibgas.to_csv('../data/Gas_SP_MIBGAS_SpotD_COMBINED.csv')

#### Extracting Data for 2023-2024
Finally, we extract data for the years 2023 and 2024 to focus on recent trends in gas prices.

In [46]:
# Filter data for the years 2023 and 2024
df_mibgas['Delivery Day'] = pd.to_datetime(df_mibgas['Delivery Day'], format='%d/%m/%Y')

df_gas_2023_2024 = df_mibgas[(df_mibgas['Delivery Day'] >= '2023-01-01') & (df_mibgas['Delivery Day'] <= '2024-12-31')]

# Saved the filtered dataset to a CSV file
#df_gas_2023_2024.to_csv('../data/Gas_SP_MIBGAS_SpotD_2023-2024.csv')

# Saved previous Excel files converted to CSVs
#df_gas2019.to_csv('../data/Gas_SP_MIBGAS_SpotD_2019-2020.csv')
#df_gas2023.to_csv('../data/Gas_SP_MIBGAS_SpotD_2023.csv')

# Display the filtered dataset
df_mibgas

,Delivery Day,MIBGAS Price Index - ES,MIBGAS Volume - ES
0,2019-01-01 06:00:00,24.20,NaN
1,2019-01-02 06:00:00,24.10,NaN
2,2019-01-03 06:00:00,24.25,NaN
3,2019-01-04 06:00:00,25.50,NaN
4,2019-01-05 06:00:00,25.00,NaN
...,...,...,...
1242,2024-05-27 00:00:00,34.70,276842.0
1243,2024-05-28 00:00:00,34.57,300249.0
1244,2024-05-29 00:00:00,34.42,273685.0
1245,2024-05-30 00:00:00,34.32,258777.0


## Electricity Prices Analysis
### Electricity Prices Analysis I: REE

#### Fetching and Combining Demand and Price Data

This section defines a function to fetch demand and price data from the REE API for a specified date range, combines the data into a single DataFrame, and saves it to a CSV file. The data is fetched for each day within the range and combined to ensure complete coverage.

In [52]:
def get_demand_and_price(demand_url, price_url, start_date, end_date, output_file):
    current_date = start_date
    all_data = []

    while current_date <= end_date:
        # Construct the URL for the current day
        day = current_date.strftime("%Y-%m-%d")
        next_day = current_date + pd.Timedelta(days=1)
        next_day_str = next_day.strftime("%Y-%m-%d")
        print(f"Getting data for {day}")

        demand_day_url = f"{demand_url}?start_date={day}T00:00&end_date={day}T23:59&time_trunc=hour&geo_trunc=electric_system&geo_limit=peninsular&geo_ids=8741"
        price_day_url = f"{price_url}?start_date={day}T00:00&end_date={day}T23:59&time_trunc=hour"

        # Make the API request for demand
        demand_response = requests.get(demand_day_url)
        if demand_response.status_code != 200:
            print(f"Error: {demand_response.json()['errors'][0]['detail']}")
            return
        demand_data = demand_response.json()
        demand_values = {item['datetime']: item['value'] for item in demand_data['included'][0]['attributes']['values']}

        # Make the API request for prices
        price_response = requests.get(price_day_url)
        if price_response.status_code != 200:
            print(f"Error: {price_response.json()['errors'][0]['detail']}")
            return
        price_data = price_response.json()
        price_values = {item['datetime']: item['value'] for item in price_data['included'][0]['attributes']['values']}

        # Combine demand and price data for the current day
        for dt in demand_values:
            all_data.append({
                'datetime': dt,
                'demand_value': demand_values[dt],
                'price_value': price_values.get(dt)  # Use .get() to handle missing price values gracefully
            })

        # Move to the next day
        current_date = next_day

    # Convert collected data to DataFrame
    combined_df = pd.DataFrame(all_data)

    # Save to CSV
    combined_df.to_csv(output_file, index=False)
    print(f"Data successfully fetched and saved to {output_file}")

In [ ]:
demand_url = 'https://apidatos.ree.es/en/datos/demanda/evolucion'
price_url = 'https://apidatos.ree.es/en/datos/mercados/precios-mercados-tiempo-real'
start_date = pd.to_datetime('2019-01-01')
end_date = pd.to_datetime('2024-05-31')
output_file = 'demand_and_price_data.csv'

get_demand_and_price(demand_url, price_url, start_date, end_date, output_file)

#### Loading and Displaying Combined Demand and Price Data
This section loads the combined demand and price data from the previously saved CSV file and displays its structure and contents. It also filters the data for the years 2019-2020 and 2023-2024, and saves these filtered datasets for further analysis.

In [53]:
df_redelectrica = pd.read_csv('../data/demand_and_price_data.csv')

# Display the DataFrame
print(df_redelectrica.head())
print()
print(df_redelectrica.info())

                        datetime  demand_value  price_value
0  2019-01-01T00:00:00.000+01:00     23209.473        66.88
1  2019-01-01T01:00:00.000+01:00     22406.330        66.88
2  2019-01-01T02:00:00.000+01:00     21090.596        66.00
3  2019-01-01T03:00:00.000+01:00     19896.113        63.64
4  2019-01-01T04:00:00.000+01:00     19146.631        58.85

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47471 entries, 0 to 47470
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      47471 non-null  object 
 1   demand_value  47471 non-null  float64
 2   price_value   47471 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.1+ MB
None


In [ ]:
# Convert 'datetime' column to datetime format and extract year
df_redelectrica['datetime'] = pd.to_datetime(df_redelectrica['datetime'])
df_redelectrica['year'] = df_redelectrica['datetime'].dt.year

# Filter the data for the years 2019-2020 and 2023-2024
df_2019_2020 = df_redelectrica[df_redelectrica['year'].isin([2019, 2020])]
df_2023_2024 = df_redelectrica[df_redelectrica['year'].isin([2023, 2024])]

#df_2019_2020.to_csv('../data/Electricity_REData_2019-2020.csv', index=False)
#df_2023_2024.to_csv('../data/Electricity_REData_2023-2024.csv', index=False)

### Electricity Prices Analysis II: OMIE

This section defines a class for importing OMIE marginal price data from the provided URLs, processing the data to ensure correct formatting, and handling any errors during the data retrieval process.

In [19]:
class OMIEMarginalPriceFileImporter:
    def __init__(self, date_ini, date_end):
        self.date_ini = date_ini
        self.date_end = date_end
        self.base_url = "https://www.omie.es/sites/default/files/dados"

    def read_to_dataframe(self, verbose=False):
        df_list = []
        for date in pd.date_range(self.date_ini, self.date_end, freq='D'):
            url = f"{self.base_url}/AGNO_{date.year}/MES_{date.month:02d}/TXT/INT_PBC_EV_H_1_{date.day:02d}_{date.month:02d}_{date.year}_{date.day:02d}_{date.month:02d}_{date.year}.TXT"
            if verbose:
                print(f"Requesting {url}")
            try:
                response = requests.get(url)
                response.raise_for_status()
                data = StringIO(response.text)
                # Read the data, skipping initial metadata rows if necessary
                df = pd.read_csv(data, delimiter=';', skiprows=1, nrows=1, index_col=False)
                df.insert(0, 'DATE', date.strftime('%Y-%m-%d'))
                df_list.append(df)
            except Exception as e:
                if verbose:
                    print(f"There was an error processing file: {url}")
                    print(e)
        # Concatenate all DataFrames if any were read successfully
        if df_list:
            df_power = pd.concat(df_list, ignore_index=True)
        else:
            df_power = pd.DataFrame(columns=columns)
        # Drop columns with all NaN values
        df_power.dropna(axis=1, how='all', inplace=True)
        # Remove any remaining unnamed columns
        df_power = df_power.loc[:, ~df_power.columns.str.contains('^Unnamed')]
        # Explicitly drop the column named '25' if it exists
        if '25' in df.columns:
            df.drop(columns=['25'], inplace=True)
        return df_power

This section sets the date range, creates an instance of the importer, reads the data into a dataframe, sorts it by date, and removes any unnecessary columns.

In [ ]:
# Set date range
dateIni = dt.datetime(2019, 1, 1)
dateEnd = dt.datetime(2024, 6, 1)

# Create an importer instance and read data
importer = OMIEMarginalPriceFileImporter(date_ini=dateIni, date_end=dateEnd)
#df = importer.read_to_dataframe(verbose=True) - 2min executing
df_power = importer.read_to_dataframe()

# Check if DataFrame has a 'DATE' column before sorting
if 'DATE' in df_power.columns:
    df_power.sort_values(by='DATE', axis=0, inplace=True)

df_power

#### Cleaning Electricity Price Data

This section reads the cleaned data from a CSV file for further analysis.

In [48]:
#df_power.to_csv('power-prices_MWh.csv')

file_path = '../data/power-prices_MWh.csv'
df_power_prices = pd.read_csv(file_path)

df_power_prices

,Unnamed: 0,DATE,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,0,2019-01-01,"66,88","66,88","66,00","63,64","58,85","55,47","56,00","61,09",...,"65,01","60,84","61,09","63,08","66,66","67,06","67,26","68,07","67,08","64,03"
1,1,2019-01-02,"61,74","53,00","48,23","47,23","45,24","45,70","51,10","58,37",...,"61,50","60,69","62,19","67,10","69,30","69,25","68,07","68,01","66,66","64,28"
2,2,2019-01-03,"62,01","54,85","52,19","50,21","50,20","52,19","54,91","66,58",...,"67,00","68,01","68,99","70,00","71,10","70,55","68,38","67,83","66,10","66,88"
3,3,2019-01-04,"67,20","63,75","58,89","54,94","54,85","58,00","63,00","68,40",...,"69,15","68,40","68,40","69,15","71,06","71,94","70,20","68,71","68,68","68,17"
4,4,2019-01-05,"71,68","69,50","68,07","66,00","63,93","62,00","60,84","62,92",...,"61,57","59,40","58,31","67,20","69,00","68,71","68,07","65,38","65,76","62,80"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,1974,2024-05-28,"35,00","35,00","35,01","35,01","35,01","35,01","41,22","51,79",...,"0,00","0,00","0,01","5,00","8,00","29,60","41,10","70,96","55,94","41,46"
1975,1975,2024-05-29,"54,77","40,94","40,58","40,34","40,70","40,94","59,36","65,00",...,"17,47","11,98","11,00","19,76","29,38","35,01","40,94","67,94","69,87","63,95"
1976,1976,2024-05-30,"50,94","45,32","43,12","40,53","38,64","42,79","45,44","45,44",...,"0,00","0,00","0,00","0,00","1,00","7,00","37,50","35,01","37,92","30,33"
1977,1977,2024-05-31,"10,00","5,50","4,77","3,25","3,25","4,37","6,67","5,00",...,"-0,01","-0,01","-0,01","-0,01","0,00","0,00","3,25","7,30","5,50","3,25"


## Weather Data Analysis

This section sets up the Open Meteo API client, fetches weather data for a specified location and date range, processes the data, and saves it to a CSV file.

In [23]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 40.4168,  # Madrid latitude
    "longitude": -3.7038,  # Madrid longitude
    "start_date": "2019-01-01",  # Start date for weather data
    "end_date": "2024-06-01",  # End date for weather data
    "hourly": "temperature_2m"  # Requested weather variable
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.to_csv('../data/weather_data.csv', index=False)
print(hourly_dataframe)

Coordinates 40.38664245605469°N -3.67608642578125°E
Elevation 666.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m
0     2019-01-01 00:00:00+00:00        0.905000
1     2019-01-01 01:00:00+00:00       -0.295000
2     2019-01-01 02:00:00+00:00       -0.745000
3     2019-01-01 03:00:00+00:00       -1.145000
4     2019-01-01 04:00:00+00:00       -1.695000
...                         ...             ...
47491 2024-06-01 19:00:00+00:00       23.355000
47492 2024-06-01 20:00:00+00:00       21.305000
47493 2024-06-01 21:00:00+00:00       19.905001
47494 2024-06-01 22:00:00+00:00       18.355000
47495 2024-06-01 23:00:00+00:00       16.905001

[47496 rows x 2 columns]


#### Reading and Filtering Weather Data

This section reads the weather data from a CSV file, converts the date column to datetime format, and filters the data for specific date ranges.

In [51]:
file_path ='../data/weather_data.csv'
df_weather = pd.read_csv(file_path)

df_weather['date'] = pd.to_datetime(df_weather['date'])

df_weather_2019_2020 = df_weather[(df_weather['date'] >= '2019-01-01') & (df_weather['date'] <= '2020-12-31')]
df_weather_2023_2024 = df_weather[(df_weather['date'] >= '2023-01-01') & (df_weather['date'] <= '2024-12-31')]

#df_weather_2019_2020.to_csv('../data/Weather_2019-2020.csv', index=False)
#df_weather_2023_2024.to_csv('../data/Weather_2023-2024.csv', index=False)

,date,temperature_2m
35064,2023-01-01 00:00:00+00:00,7.155
35065,2023-01-01 01:00:00+00:00,6.805
35066,2023-01-01 02:00:00+00:00,6.155
35067,2023-01-01 03:00:00+00:00,4.055
35068,2023-01-01 04:00:00+00:00,3.705
...,...,...
47491,2024-06-01 19:00:00+00:00,23.355
47492,2024-06-01 20:00:00+00:00,21.305
47493,2024-06-01 21:00:00+00:00,19.905
47494,2024-06-01 22:00:00+00:00,18.355
